In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import pyro
import pyro.distributions as dist
import pyro.infer.mcmc as mcmc

from path import Path

latent_dim_dict = {
    'Arizona':5, 
    'Atlanta':7 , 
    'Baltimore':15, 
    'Buffalo':25, 
    'Carolina':5, 
    'Chicago':15, 
    'Cincinnati':13, 
    'Cleveland':1, 
    'Dallas':37,
    'Denver':5 , 
    'Detroit':12, 
    'Green Bay':52, 
    'Houston':12, 
    'Indianapolis':10, 
    'Jacksonville':12, 
    'Kansas City':11, 
    'Las Vegas':12, 
    'LA Chargers':5,
    'LA Rams':8, 
    'Miami':8, 
    'Minnesota':5, 
    'New England':9, 
    'New Orleans':18 , 
    'NY Giants':7,
    'NY Jets':7, 
    'Philadelphia':10, 
    'Pittsburgh':25, 
    'San Francisco':10, 
    'Seattle':14, 
    'Tampa Bay':15, 
    'Tennessee':13, 
    'Washington':4
                   }

In [9]:
class PPCA:
    """Probabilistic PCA (PPCA)"""

    def __init__(self, latent_dim, variance_support=(0, 10)):
        self.latent_dim = latent_dim
        self.variance_support = variance_support
    
    def likelihood(self, **kwargs):
        H = kwargs['H']
        Z = kwargs['Z']
        # this indexing handles the possibility of posterior samples
        sigma = kwargs['sigma'][..., None, None]
        return dist.Normal(H @ Z, sigma)
    
    def model(self, data, mask=None):
        n_time, n_unit = data.shape

        # Sample latent factors (K x N)
        Z = pyro.sample('Z', dist.Normal(torch.zeros(self.latent_dim, n_unit), 1.).to_event(2))

        # Sample weights (T x K)
        H = pyro.sample('H', dist.Normal(torch.zeros(n_time, self.latent_dim), 1.).to_event(2))

        # Sample observed variance
        sigma = pyro.sample("sigma", dist.Uniform(self.variance_support[0], self.variance_support[1]))

        # Sample observed data
        with pyro.plate("data"):
            likelihood = self.likelihood(H=H, Z=Z, sigma=sigma)
            if mask is not None:
                likelihood = likelihood.mask(mask)
            Y = pyro.sample("Y", likelihood, obs=data)

        return Y, Z, H, sigma

class GAP:
    """The Gamma-Poisson (GaP) factor model"""

    def __init__(self, latent_dim, shp=1., rte=1.):
        self.latent_dim = latent_dim
        self.shp = shp
        self.rte = rte
    
    def likelihood(self, **kwargs):
        return dist.Poisson(kwargs['H'] @ kwargs['Z'])
    
    def model(self, data, mask=None):
        n_time, n_unit = data.shape
        shp, rte, latent_dim = self.shp, self.rte, self.latent_dim

        # Sample latent factors (K x N)
        Z = pyro.sample("Z", dist.Gamma(concentration=shp, rate=rte).expand([latent_dim, n_unit]).to_event(2))

        # Sample weights (T x K)
        H = pyro.sample("H", dist.Gamma(concentration=shp, rate=rte).expand([n_time, latent_dim]).to_event(2))

        # Sample observed data
        with pyro.plate("data"):
            likelihood = self.likelihood(H=H, Z=Z)
            if mask is not None:
                likelihood = likelihood.mask(mask)
            Y = pyro.sample("Y", likelihood, obs=data)
        
        return Y, Z, H

def run_NUTS_with_mask(model, data, mask, warmup_steps, num_samples):
    """Runs NUTS with given model, data, mask and returns posterior samples."""
    
    pyro.clear_param_store() # do we need this?

    # Define MCMC kernel
    kernel = mcmc.NUTS(model)
    mcmc_run = mcmc.MCMC(kernel, num_samples=num_samples, warmup_steps=warmup_steps)

    # Run MCMC process on our data with given latent dimension and mask
    mcmc_run.run(data, mask)

    # Extract samples
    posterior_samples = mcmc_run.get_samples()
    
    return posterior_samples

In [11]:
def population_predictive_check(data, mask, model, posterior_samples):
    model_name = model.__class__.__name__
    posterior_predictive_samples = model.likelihood(**posterior_samples).sample()

    log_prob_fake = model.likelihood(**posterior_samples).log_prob(posterior_predictive_samples)
    log_prob_true = model.likelihood(**posterior_samples).log_prob(data)

    d_fake = -log_prob_fake[:, ~mask].sum(axis=-1)
    d_true = -log_prob_true[:, ~mask].sum(axis=-1)

    ppop = (d_fake > d_true).float().mean()
    return ppop

In [4]:
def create_mask(data, mask_type="Random"):
    assert mask_type in ["Random", "Speckled", "End", "None"]

    n_time, n_unit = data.shape
    mask = torch.ones(n_time, n_unit)

    if mask_type == "Random":
        mask_rows = torch.randperm(n_time)[:20]
        mask_cols = torch.randperm(n_unit)[:5]
        for t in mask_rows:
            for i in mask_cols:
                mask[t, i] = 0
    
    elif mask_type == "Speckled":
        # hold out 1% of the data
        mask = torch.bernoulli(mask * 0.99)
        
    elif mask_type == "End":
        mask[-30:, -3:] = 0
    
    mask = mask.bool()
    return mask

In [17]:
# Define MCMC parameters
warmup_steps = 1000
num_samples = 2000

# Data directory (must already exist)
dat_path = Path('dat')
assert dat_path.exists()

# Traverse through directory and find training data tensors
for train_data_file in dat_path.walkfiles('train_data.pt'):
    # Get team name from directory
    team = train_data_file.parent.splitpath()[-1]
    print('\n----------------------------------------')

    # Load training data tensor
    train_data = torch.load(train_data_file)

    # Get latent dimension
    latent_dim = latent_dim_dict[team]

    # Define model classes
    models = [GAP(latent_dim=latent_dim, shp=1.0, rte=1.0),
              PPCA(latent_dim=latent_dim, variance_support=(0, 10))]

    # Create a results directory
    out_path = Path('out').joinpath(team)
    out_path.makedirs_p()

    # Create and save mask (use .pt extension for tensor)
    mask = create_mask(train_data, mask_type="End")
    torch.save(mask, out_path.joinpath('mask.pt'))

    # Run MCMC for each model
    for model in models:
        model_name = model.__class__.__name__
        print(f"Running {model_name} for {team}...")

        posterior_samples = run_NUTS_with_mask(model=model.model, 
                                               data=train_data,
                                               mask=mask,
                                               warmup_steps=warmup_steps,
                                               num_samples=num_samples)
        
        # Save posterior samples (use .pth extension for dict of tensors)
        torch.save(posterior_samples, out_path.joinpath('posterior_samples.pth'))

        # Generate and save posterior predictive samples
        posterior_predictive_samples = model.likelihood(**posterior_samples).sample()
        assert posterior_predictive_samples.shape == (num_samples,) + train_data.shape
        torch.save(posterior_predictive_samples, out_path.joinpath('posterior_predictive_samples.pt'))

        # Run population predictive check (this will re-generate posterior predictive samples)
        ppop = population_predictive_check(data=train_data, 
                                           mask=mask, 
                                           model=model,
                                           posterior_samples=posterior_samples)
        print(f"Population predictive check p-value: {ppop:.3f}\n")


----------------------------------------
Running GAP for Detroit...


Warmup:  10%|▉         | 285/3000 [01:02,  1.13it/s, step size=7.83e-04, acc. prob=0.782]

KeyboardInterrupt: 

In [14]:
out_path.makedirs_p()

Path('out/Detroit')

In [30]:
# Traverse through directory and find training data tensors
for train_data_file in dat_path.walkfiles('train_data.pt'):
    # Get team name from directory
    team = train_data_file.parent.splitpath()[-1]
    if team != 'Baltimore':
        continue

    # Load training data tensor
    train_data = torch.load(train_data_file)

mask = create_mask(train_data, mask_type="End")

Z = np.load('/Users/aaronschein/Downloads/Baltimore_g_samples.npy')
H = np.load('/Users/aaronschein/Downloads/Baltimore_f_samples.npy')
posterior_samples = {'Z': torch.from_numpy(Z), 'H': torch.from_numpy(H)}

n_time, n_unit = train_data.shape
latent_dim = latent_dim_dict['Baltimore']
num_samples = Z.shape[0]

assert H.shape == (num_samples, n_time, latent_dim)
assert Z.shape == (num_samples, latent_dim, n_unit)

model = GAP(latent_dim=latent_dim, shp=1.0, rte=1.0)

ppop = population_predictive_check(data=train_data, 
                                   mask=mask, 
                                   model=model,
                                   posterior_samples=posterior_samples)

ppop

tensor(0.)

ls ~/Downloads

ls 